In [2]:
"""==========Solving job shop scheduling problem by gentic algorithm in python======="""
# importing required modules
import pandas as pd
import numpy as np
import time
import copy
from colorama import init
from termcolor import colored

In [4]:
""" ================= initialization setting ======================"""


pt_tmp = pd.read_excel("data/la16.xlsx", sheet_name="Processing time", index_col=[0])
machine_tmp = pd.read_excel(
    "data/la16.xlsx", sheet_name="Machine Numbers", index_col=[0]
)

num_job = pt_tmp.shape[0]  # number of jobs
num_mc = 10  # number of machines
pt = [list(map(int, pt_tmp.iloc[i])) for i in range(num_job)]
jt = [list(map(int, jt_tmp.iloc[i])) for i in range(num_job)]
print(pt)
print(num_job)

[[21, 71, 16, 52, 26, 34, 53, 21, 55, 95], [55, 31, 98, 79, 12, 66, 42, 77, 77, 39], [34, 64, 62, 19, 92, 79, 43, 54, 83, 37], [87, 69, 87, 38, 24, 83, 41, 93, 77, 60], [98, 44, 25, 75, 43, 49, 96, 77, 17, 79], [35, 76, 28, 10, 61, 9, 95, 35, 7, 95], [16, 59, 46, 91, 43, 50, 52, 59, 28, 27], [45, 87, 41, 20, 54, 43, 14, 9, 39, 71], [33, 37, 66, 33, 26, 8, 28, 89, 42, 78], [69, 81, 94, 96, 27, 69, 45, 78, 74, 84]]
10


In [ ]:
pt = [list(map(int, pt_tmp.iloc[i][:3])) for i in range(num_job)]
# setup = list(map(float, machine_tmp.loc["SETUP"]))  # 換線時間
orderSequnce = list(pt_tmp.index[:-1])

population_size = 30
crossover_rate = 0.8
mutation_rate = 0.2
mutation_selection_rate = 0.2
num_mutation_jobs = round(num_gene * mutation_selection_rate)
num_iteration = 2000

start_time = time.time()
pt
# setup
# orderSequnce

In [ ]:
"""==================== main code ==============================="""
"""----- generate initial population -----"""
Tbest = 999999999999999
best_list, best_obj = [], []
population_list = []
makespan_record = []
for i in range(population_size):
    random_chromosome = [
        np.random.permutation(
            np.arange(1, num_gene + 1) % num_job
        ).tolist(),  # operation sequence
        np.random.randint(1, num_mc + 1, size=num_gene).tolist(),  # machine selection
    ]
    population_list.append(random_chromosome)


len(population_list[0][1])
# len(population_list[0])
# len(population_list)
# len(population_list[0][0])
population_list[0][0]

In [ ]:
def relativeSortArray(A, B, cutpoint):
    new_A = copy.deepcopy(A)
    # sort OS according to the sequence in B
    new_A[0][cutpoint:] = sorted(
        new_A[0][cutpoint:], key=(B + sorted(new_A[0][cutpoint:])).index
    )
    # print(len(new_A[0]))
    # print(new_A[0])
    # sort MS according to how OS is sorted
    indices = [A[0].index(element) - 1 for element in new_A[0]]
    temp = np.array(new_A[1])
    new_A[1] = temp[indices].tolist()
    # mutate (choose a candidate with largest reference point)
    mutation_prob_1 = np.random.rand()
    if mutation_rate >= mutation_prob_1:
        new_A[0][cutpoint:] = np.roll(new_A[0][cutpoint:], 1)
        new_A[1][cutpoint:] = np.roll(new_A[1][cutpoint:], 1)
    # mutate (random machine selection)
    mutation_prob_2 = np.random.rand()
    if mutation_rate >= mutation_prob_2:
        op_to_be_changed = np.random.randint(0, len(new_A[1]))
        new_A[1][op_to_be_changed] = np.random.randint(1, num_mc + 1)
    return new_A

In [ ]:
for n in range(num_iteration):
    Tbest_now = 99999999999

    """-------- two point crossover --------"""
    parent_list = copy.deepcopy(population_list)
    offspring_list = copy.deepcopy(population_list)
    S = list(
        np.random.permutation(population_size)
    )  # generate a random sequence to select the parent chromosome to crossover

    for m in range(int(population_size / 2)):
        crossover_prob = np.random.rand()
        if crossover_rate >= crossover_prob:
            parent_1 = population_list[S[2 * m]][:]
            parent_2 = population_list[S[2 * m + 1]][:]
            child_1 = parent_1[:]
            child_2 = parent_2[:]
            cutpoint = np.random.randint(0, num_gene)

            child_1 = relativeSortArray(child_1, child_2[0], cutpoint)
            child_2 = relativeSortArray(child_2, child_1[0], cutpoint)

            offspring_list[S[2 * m]] = child_1[:]
            offspring_list[S[2 * m + 1]] = child_2[:]

    """--------fitness value(calculate makespan)-------------"""
    total_chromosome = copy.deepcopy(parent_list) + copy.deepcopy(
        offspring_list
    )  # parent and offspring chromosomes combination
    chrom_fitness, chrom_fit = [], []
    total_fitness = 0
    for m in range(population_size * 2):  # iterate through all chromosomes
        j_keys = [j for j in range(num_job)]
        # number of tasks(工序) completed for each job
        key_count = {key: 0 for key in j_keys}
        # track the cumulative processing time for each job
        j_count = {key: 0 for key in j_keys}

        m_keys = [j for j in range(num_mc)]
        # track the cumulative processing time for each machine
        m_count = {key: 0 for key in m_keys}

        """ new added """
        for job, machine in zip(total_chromosome[m][0], total_chromosome[m][1]):
            processing_time = int(pt[job][key_count[job]])
            print(job, machine)
            type = job_type[machine - 1]
            setup_time = setup_time_table[machine - 1]
            job_type = [
                [] for _ in range(num_mc)
            ]  # record the sequence of job type of each machine

            if np.size(job_type[machine - 1]) == 0:  # first job on this machine
                j_count[job] += setup_time + processing_time
                m_count[machine - 1] += processing_time

                # time[machine - 1] += setup_time + processing_time
                job_type[machine - 1].append(type)
            else:
                # get the last job type on this machine to determine if setup is needed
                last_type = job_type[machine - 1][-1]
                if type == last_type:
                    j_count[job] += processing_time
                    # time[machine - 1] += processing_time
                else:
                    j_count[job] += setup_time + processing_time
                    # time[machine - 1] += setup_time + processing_time
                job_type[machine - 1].append(type)

            if m_count[machine - 1] < j_count[job]:
                m_count[machine - 1] = j_count[job]
            elif m_count[machine - 1] > j_count[job]:
                j_count[job] = m_count[machine - 1]

            key_count[job] = key_count[job] + 1

        makespan = max(j_count.values())
        chrom_fitness.append(1 / makespan)
        chrom_fit.append(makespan)
        total_fitness = total_fitness + chrom_fitness[m]

    """----------selection(roulette wheel approach)----------"""
    pk, qk = [], []

    for i in range(population_size * 2):
        pk.append(chrom_fitness[i] / total_fitness)
    for i in range(population_size * 2):
        cumulative = 0
        for j in range(0, i + 1):
            cumulative = cumulative + pk[j]
        qk.append(cumulative)

    selection_rand = [np.random.rand() for i in range(population_size)]

    for i in range(population_size):
        if selection_rand[i] <= qk[0]:
            population_list[i] = copy.deepcopy(total_chromosome[0])
        else:
            for j in range(0, population_size * 2 - 1):
                if selection_rand[i] > qk[j] and selection_rand[i] <= qk[j + 1]:
                    population_list[i] = copy.deepcopy(total_chromosome[j + 1])
                    break
    """----------comparison----------"""
    for i in range(population_size * 2):
        if chrom_fit[i] < Tbest_now:
            Tbest_now = chrom_fit[i]
            sequence_now = copy.deepcopy(total_chromosome[i])
    if Tbest_now <= Tbest:
        Tbest = Tbest_now
        sequence_best = copy.deepcopy(sequence_now)

    makespan_record.append(Tbest)

In [ ]:
"""----------result----------"""
import matplotlib.pyplot as plt

print("optimal sequence", sequence_best)
print("optimal value:%f" % Tbest)
print("the elapsed time:%s" % (time.time() - start_time))


# %matplotlib inline
plt.plot([i for i in range(len(makespan_record))], makespan_record, "b")
plt.ylabel("makespan", fontsize=15)
plt.xlabel("generation", fontsize=15)
plt.show()

In [ ]:
"""--------plot gantt chart-------"""
import pandas as pd
import chart_studio as cs
import chart_studio.plotly as py
import plotly.figure_factory as ff
import datetime

m_keys = [j + 1 for j in range(num_mc)]
j_keys = [j for j in range(num_job)]
key_count = {key: 0 for key in j_keys}
j_count = {key: 0 for key in j_keys}
m_count = {key: 0 for key in m_keys}
j_record = {}
for i in sequence_best:
    gen_t = int(pt[i][key_count[i]])
    gen_m = int(ms[i][key_count[i]])
    j_count[i] = j_count[i] + gen_t
    m_count[gen_m] = m_count[gen_m] + gen_t

    if m_count[gen_m] < j_count[i]:
        m_count[gen_m] = j_count[i]
    elif m_count[gen_m] > j_count[i]:
        j_count[i] = m_count[gen_m]

    start_time = str(
        datetime.timedelta(seconds=j_count[i] - pt[i][key_count[i]])
    )  # convert seconds to hours, minutes and seconds
    end_time = str(datetime.timedelta(seconds=j_count[i]))

    j_record[(i, gen_m)] = [start_time, end_time]

    key_count[i] = key_count[i] + 1


df = []
for m in m_keys:
    for j in j_keys:
        df.append(
            dict(
                Task="Machine %s" % (m),
                Start="2018-07-14 %s" % (str(j_record[(j, m)][0])),
                Finish="2018-07-14 %s" % (str(j_record[(j, m)][1])),
                Resource="Job %s" % (j + 1),
            )
        )

fig = ff.create_gantt(
    df,
    index_col="Resource",
    show_colorbar=True,
    group_tasks=True,
    showgrid_x=True,
    title="Job shop Schedule",
)
py.iplot(fig, filename="GA_job_shop_scheduling", world_readable=True)